# CodonScope: Multi-Species Codon Usage Analysis

Analyze codon usage patterns in your gene lists across yeast, human, and mouse.

CodonScope runs 6 analysis modes:

| Mode | Name | What it does |
|------|------|--------------|
| 1 | **Composition** | Compares mono/di/tricodon frequencies in your genes vs the genome. Finds over- or under-represented codons. |
| 2 | **Translational Demand** | Weights codon usage by expression level (TPM). Shows which codons are under translational selection pressure. |
| 3 | **Optimality Profile** | Plots per-position codon optimality (tAI/wtAI) along transcripts. Detects 5' ramp regions. |
| 4 | **Collision Potential** | Counts fast-to-slow (FS) codon transitions that may cause ribosome collisions. |
| 5 | **Disentanglement** | Separates amino acid composition effects from synonymous codon choice (RSCU). Classifies drivers as tRNA supply, GC3 bias, or wobble avoidance. |
| 6 | **Cross-Species** | Correlates RSCU between ortholog pairs across species. Identifies genes with divergent codon preferences. |

**How to use this notebook:**
1. Run cells 1-2 to install CodonScope and download reference data (first run takes ~5 min)
2. Paste your gene list in cell 3
3. Run cells 4-5 to see results

## 1. Install CodonScope

In [ ]:
#@title Install CodonScope and dependencies { display-mode: "form" }

import subprocess, sys, os

# ── Configuration ────────────────────────────────────────────────────────────
GITHUB_REPO = "https://github.com/Meier-Lab-NCI/codonscope.git"
# ──────────────────────────────────────────────────────────────────────────────

def run(cmd):
    """Run a shell command and print output."""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.returncode != 0 and result.stderr.strip():
        print(result.stderr.strip())
    return result.returncode

# Install dependencies
print("Installing dependencies...")
run(f"{sys.executable} -m pip install -q numpy scipy pandas matplotlib requests tqdm")

# Install CodonScope from GitHub
print(f"\nInstalling CodonScope from GitHub...")
run(f"{sys.executable} -m pip install -q git+{GITHUB_REPO}")

# Enable ipywidgets in Colab (required for Textarea rendering)
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("Enabled Colab custom widget manager for interactive inputs")
except Exception:
    pass

# Verify installation
try:
    import codonscope
    print(f"\n✓ CodonScope v{codonscope.__version__} installed successfully")
except ImportError:
    print("\n✗ Installation failed. Check the GitHub URL above.")

## 2. Download Reference Data

Downloads CDS sequences, tRNA copy numbers, wobble rules, expression data, and pre-computed backgrounds for the selected species.

If Google Drive is mounted, data is cached there so you only download once across sessions. Otherwise data is stored in the Colab runtime (re-downloaded each session).

In [ ]:
#@title Download species data (cached to Google Drive) { display-mode: "form" }

import os, shutil
from pathlib import Path

# ── Which species to download ─────────────────────────────────────────────────
download_human = True  #@param {type:"boolean"}
download_yeast = True  #@param {type:"boolean"}
download_mouse = False  #@param {type:"boolean"}

# Also download orthologs for cross-species comparison (Mode 6)?
DOWNLOAD_ORTHOLOGS = False  #@param {type:"boolean"}
# ──────────────────────────────────────────────────────────────────────────────

SPECIES = []
if download_human:
    SPECIES.append("human")
if download_yeast:
    SPECIES.append("yeast")
if download_mouse:
    SPECIES.append("mouse")

if not SPECIES:
    raise ValueError("Select at least one species to download.")

# Try to mount Google Drive for persistent caching
USE_DRIVE = False
DRIVE_DATA_DIR = Path("/content/drive/MyDrive/codonscope_data")
LOCAL_DATA_DIR = Path.home() / ".codonscope" / "data"

try:
    from google.colab import drive
    if not os.path.exists("/content/drive/MyDrive"):
        print("Mounting Google Drive for persistent data caching...")
        drive.mount("/content/drive")
    USE_DRIVE = True
    print("✓ Google Drive mounted — data will be cached between sessions")
except (ImportError, Exception) as e:
    print(f"Google Drive not available ({e}). Data will be stored locally.")

if USE_DRIVE:
    # Symlink so codonscope reads from Drive
    DRIVE_DATA_DIR.mkdir(parents=True, exist_ok=True)
    LOCAL_DATA_DIR.parent.mkdir(parents=True, exist_ok=True)
    if LOCAL_DATA_DIR.is_symlink():
        LOCAL_DATA_DIR.unlink()
    elif LOCAL_DATA_DIR.exists():
        shutil.rmtree(LOCAL_DATA_DIR)
    LOCAL_DATA_DIR.symlink_to(DRIVE_DATA_DIR)
    print(f"  Data directory: {DRIVE_DATA_DIR}")

# Download each species (skips if data already exists)
from codonscope.data.download import download, download_orthologs

for sp in SPECIES:
    species_dir = LOCAL_DATA_DIR / "species" / sp
    marker = species_dir / "background_mono.npz"
    if marker.exists():
        n_files = len(list(species_dir.glob("*")))
        print(f"\n✓ {sp} data already cached ({n_files} files). Skipping download.")
    else:
        print(f"\nDownloading {sp} data (this may take a few minutes)...")
        download(sp)
        print(f"✓ {sp} data downloaded")

# Optionally download orthologs
if DOWNLOAD_ORTHOLOGS and len(SPECIES) >= 2:
    ortho_dir = LOCAL_DATA_DIR / "orthologs"
    pairs = []
    if "human" in SPECIES and "yeast" in SPECIES:
        pairs.append(("human", "yeast"))
    if "human" in SPECIES and "mouse" in SPECIES:
        pairs.append(("human", "mouse"))
    if "mouse" in SPECIES and "yeast" in SPECIES:
        pairs.append(("mouse", "yeast"))
    for s1, s2 in pairs:
        ortho_file = ortho_dir / f"{s1}_{s2}.tsv"
        if ortho_file.exists():
            print(f"\n✓ {s1}-{s2} orthologs already cached. Skipping.")
        else:
            print(f"\nDownloading {s1}-{s2} orthologs...")
            download_orthologs(s1, s2)
            print(f"✓ {s1}-{s2} orthologs downloaded")

print("\n" + "="*50)
print("All data ready. Proceed to enter your gene list.")

## 3. Enter Your Gene List

Paste gene IDs into the text box below. Supported ID types:

| Species | Accepted IDs | Examples |
|---------|-------------|----------|
| **Yeast** | Systematic names, common names, SGD IDs, UniProt | `YFL039C`, `ACT1`, `SGD:S000001855`, `P60010` |
| **Human** | HGNC symbols, Ensembl (ENSG/ENST), Entrez, RefSeq, UniProt | `TP53`, `ENSG00000141510`, `7157`, `NM_000546`, `P04637` |
| **Mouse** | MGI symbols, Ensembl (ENSMUSG/ENSMUST), MGI IDs, Entrez, UniProt | `Actb`, `ENSMUSG00000029580`, `MGI:87904`, `11461` |

You can mix ID types in the same list. After pasting, run the **Confirm** cell below.

### Expression source (human only, for Mode 2)

- **Tissue dropdown**: Select a GTEx v8 tissue for tissue-specific expression. Leave blank for cross-tissue median.
- **Cell line dropdown**: Common cell lines (HEK293T, HeLa, etc.) use the closest GTEx tissue as a proxy.
- **Custom expression**: To supply your own TPM values, save a tab-separated file with two columns and load it via the CLI:
  ```
  gene_id    tpm
  TP53       12.5
  BRCA1      8.3
  MYC        150.0
  ```
  Then run: `python3 -m codonscope.cli demand --species human --genes genes.txt --expression my_tpm.tsv`

In [ ]:
#@title Enter gene list and select species { display-mode: "form" }

# ── Select species ────────────────────────────────────────────────────────────
SPECIES_NAME = "yeast"  #@param ["yeast", "human", "mouse"]

# ── Optional: cross-species comparison ────────────────────────────────────────
SPECIES2 = ""  #@param ["", "yeast", "human", "mouse"]
# Leave blank to skip Mode 6. Requires orthologs downloaded above.

# ── Human expression source (for Mode 2: Translational Demand) ───────────────
# GTEx tissue: real RNA-seq from the GTEx v8 consortium (54 normal tissues).
# Cell line: uses GTEx tissue proxy (e.g. HEK293T → Kidney Cortex).
# Leave both blank for cross-tissue median (default).
TISSUE = ""  #@param ["", "cross_tissue_median", "Liver", "Lung", "Brain - Cortex", "Brain - Cerebellum", "Heart - Left Ventricle", "Kidney - Cortex", "Muscle - Skeletal", "Pancreas", "Whole Blood", "Breast - Mammary Tissue", "Colon - Sigmoid", "Stomach", "Thyroid", "Testis", "Ovary", "Prostate", "Spleen", "Skin - Sun Exposed (Lower leg)", "Adipose - Subcutaneous", "Artery - Aorta", "Esophagus - Mucosa", "Nerve - Tibial", "Pituitary", "Small Intestine - Terminal Ileum"]
CELL_LINE = ""  #@param ["", "HEK293T", "HeLa", "K562", "A549", "MCF7", "U2OS", "HepG2", "SH-SY5Y", "Jurkat", "RPE1"]
# HEK293T → Kidney Cortex, HeLa → Uterus, K562 → Lymphocytes,
# A549 → Lung, MCF7 → Breast, HepG2 → Liver, SH-SY5Y → Brain Cortex.
# ──────────────────────────────────────────────────────────────────────────────

import ipywidgets as widgets
from IPython.display import display as ipy_display

_default_genes = """RPL3
RPL4A
RPL5
RPL7A
RPL8A
RPL9A
RPL10
RPL11A
RPL13A
RPL14A
RPL16A
RPL19A
RPL20A
RPL21A
RPL25
RPL30
RPL31A
RPL33A
RPL35A
RPL37A
RPL40A
RPL42A
RPS0A
RPS3
RPS4A
RPS5
RPS6A
RPS7A
RPS8A
RPS9A
RPS11A
RPS13
RPS14A
RPS15
RPS17A
RPS19A
RPS20
RPS21A
RPS22A
RPS24A
RPS25A
RPS26A
RPS28A
RPS29A
RPS31"""

gene_input = widgets.Textarea(
    value=_default_genes,
    placeholder="Paste gene IDs here (one per line or comma-separated)",
    description="",
    layout=widgets.Layout(width="400px", height="250px"),
)

print("Paste your gene list below (one per line or comma-separated):")
ipy_display(gene_input)
print("\n↑ Edit the box above, then run the next cell.")

In [ ]:
#@title Confirm gene list { display-mode: "form" }

import re

def parse_genes(text):
    """Parse gene list from text: one per line, comma-separated, or mixed."""
    genes = []
    for line in text.strip().split("\n"):
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        parts = re.split(r"[,\t]+", line)
        for part in parts:
            part = part.strip()
            if part:
                genes.append(part)
    return genes

gene_ids = parse_genes(gene_input.value)
species2 = SPECIES2 if SPECIES2 else None
tissue = TISSUE if TISSUE else None
cell_line = CELL_LINE if CELL_LINE else None

print(f"Species: {SPECIES_NAME}")
print(f"Gene list: {len(gene_ids)} genes")
if len(gene_ids) <= 20:
    print(f"  {', '.join(gene_ids)}")
else:
    print(f"  {', '.join(gene_ids[:10])}, ... ({len(gene_ids)-10} more)")
if species2:
    print(f"Cross-species comparison: {SPECIES_NAME} vs {species2}")
if tissue:
    print(f"Tissue: {tissue}")
if cell_line:
    print(f"Cell line: {cell_line}")
if len(gene_ids) < 10:
    print("\n⚠ Warning: fewer than 10 genes. Results may lack statistical power.")

## 4. Generate Full Report

Runs all applicable analysis modes and produces a self-contained HTML report with embedded plots.

The report includes:
- **Gene summary** — mapped/unmapped genes, CDS statistics
- **Mode 1** — Volcano plots and bar charts for mono- and dicodon composition
- **Mode 5** — Attribution table (AA-driven vs synonymous codon choice)
- **Mode 3** — Metagene optimality profile with ramp analysis
- **Mode 4** — Ribosome collision potential (FS transition analysis)
- **Mode 2** — Expression-weighted translational demand
- **Mode 6** — Cross-species RSCU correlation (if second species selected)

In [ ]:
#@title Run full report { display-mode: "form" }

import tempfile
from pathlib import Path
from IPython.display import HTML, display
from codonscope.report import generate_report

# Generate report to a temp file
output_path = Path(tempfile.mkdtemp()) / "codonscope_report.html"

print("Running all analysis modes... (this may take 1-2 minutes)")
print()

report_kwargs = dict(
    species=SPECIES_NAME,
    gene_ids=gene_ids,
    output=output_path,
    n_bootstrap=10_000,
    seed=42,
)
if species2:
    report_kwargs["species2"] = species2
if tissue:
    report_kwargs["tissue"] = tissue
if cell_line:
    report_kwargs["cell_line"] = cell_line

report_path = generate_report(**report_kwargs)
print(f"\n✓ Report generated: {report_path}")

# Display inline
html_content = report_path.read_text()
display(HTML(html_content))

# Offer download — prefer zip (includes HTML + all data TSVs)
try:
    from google.colab import files
    zip_path = report_path.parent / f"{report_path.stem}_results.zip"
    if zip_path.exists():
        print("\nDownloading results zip (HTML report + data tables)...")
        files.download(str(zip_path))
    else:
        print("\nDownloading HTML report...")
        files.download(str(report_path))
except ImportError:
    print(f"\nReport saved to: {report_path}")

## 5. Explore Individual Mode Results

Run specific modes individually and view results as sortable tables. Each mode cell has adjustable parameters explained below.

### Parameter Guide

| Parameter | Mode | Options | What it means |
|-----------|------|---------|---------------|
| **KMER_SIZE** | 1 | `1` = monocodon (single codons, 61 total), `2` = dicodon (adjacent codon pairs, 3,721 total), `3` = tricodon (codon triplets, 226,981 total) | Larger k-mers capture context-dependent effects (e.g. ribosome stalling at specific codon pairs) but require more statistical power. Start with `1`. |
| **BACKGROUND** | 1 | `all` = compare against all genes in the genome, `matched` = compare against genes matched for CDS length and GC content | Use `all` first. If GC or length bias is flagged, re-run with `matched` to see which codons remain significant after controlling for nucleotide composition. |
| **METHOD** | 3 | `wtai` = wobble-penalized tRNA Adaptation Index, `tai` = standard tAI without wobble penalty | `wtai` (default) penalises wobble base-pairing (G:U, I:C) by 0.5x because wobble decoding is slower than Watson-Crick. `tai` treats all anticodon-codon pairings equally. Use `wtai` unless you have a reason not to. |
| **RAMP_CODONS** | 3 | Number of codons from the 5' end to define the "ramp" region (default: 50) | Many highly-expressed genes have a stretch of sub-optimal codons near the start that slows initial elongation and spaces out ribosomes. 30-50 codons is typical. |
| **TISSUE** | 2 | GTEx v8 tissue name (e.g. "Liver", "Brain - Cortex") | Sets which tissue's RNA-seq TPM values weight the translational demand calculation. Different tissues express different genes, so the demand landscape changes. |
| **CELL_LINE** | 2 | Cell line name (e.g. "HEK293T") | Uses the closest GTEx tissue as a proxy for cell line expression. |

### How to interpret results

- **Z-score**: How many standard deviations your gene set differs from the genome. Positive = enriched, negative = depleted.
- **adjusted_p**: Benjamini-Hochberg FDR-corrected p-value. < 0.05 is significant after controlling for multiple testing across all codons.
- **cohens_d**: Effect size independent of sample size. |d| < 0.2 is negligible, 0.2-0.5 is small, 0.5-0.8 is medium, > 0.8 is large. A codon can be statistically significant (low p) but biologically trivial (small d).
- **FS enrichment**: Ratio of fast-to-slow codon transitions in your genes vs genome. > 1.0 = more collision-prone junctions. Highly expressed genes tend to have FS enrichment near or below 1.0.
- **RSCU**: Relative Synonymous Codon Usage. 1.0 = codon used equally with its synonyms. > 1.0 = preferred, < 1.0 = avoided. Excludes Met and Trp (single-codon amino acids).
- **wtAI**: Wobble-penalized tRNA Adaptation Index. Score from 0 to 1 based on tRNA gene copy numbers. Higher = more efficiently decoded. "Fast" codons have wtAI above the genome median, "slow" codons below.

In [ ]:
#@title Mode 1: Sequence Composition { display-mode: "form" }

import pandas as pd
from IPython.display import display, HTML
from codonscope.modes.mode1_composition import run_composition

# 1 = monocodon (61 single codons), 2 = dicodon (3,721 adjacent pairs), 3 = tricodon (226,981 triplets)
KMER_SIZE = 1  #@param [1, 2, 3] {type:"integer"}
# "all" = compare vs whole genome; "matched" = compare vs genes matched for CDS length + GC content
BACKGROUND = "all"  #@param ["all", "matched"]

kmer_labels = {1: "Monocodon", 2: "Dicodon", 3: "Tricodon"}
print(f"Running {kmer_labels[KMER_SIZE]} composition analysis ({BACKGROUND} background)...")
if BACKGROUND == "matched":
    print("  (matched = controls for CDS length and GC content differences)")

result = run_composition(
    species=SPECIES_NAME,
    gene_ids=gene_ids,
    k=KMER_SIZE,
    background=BACKGROUND,
    seed=42,
)

df = result["results"]
n = result["n_genes"]
n_sig = len(df[df["adjusted_p"] < 0.05])
unmapped = result["id_summary"].unmapped

print(f"\n{n} genes mapped, {len(unmapped)} unmapped")
if unmapped:
    print(f"  Unmapped: {', '.join(unmapped[:10])}{'...' if len(unmapped) > 10 else ''}")
print(f"{n_sig} significant {kmer_labels[KMER_SIZE].lower()}s (adjusted p < 0.05)")

# Show top results sorted by |z_score|
print(f"\nTop 20 by |Z-score|:")
top = df.head(20).copy()
top = top.round({"observed_freq": 5, "expected_freq": 5, "z_score": 2, "p_value": 2, "adjusted_p": 4, "cohens_d": 3})
display(top.style.applymap(
    lambda v: "background-color: #d4edda" if isinstance(v, float) and v < 0.05 else "",
    subset=["adjusted_p"]
))

In [ ]:
#@title Mode 2: Translational Demand { display-mode: "form" }

from codonscope.modes.mode2_demand import run_demand

# Override tissue/cell line for this mode (uses values from cell 3 if left blank)
MODE2_TISSUE = ""  #@param ["", "cross_tissue_median", "Liver", "Lung", "Brain - Cortex", "Brain - Cerebellum", "Heart - Left Ventricle", "Kidney - Cortex", "Muscle - Skeletal", "Pancreas", "Whole Blood", "Breast - Mammary Tissue", "Thyroid", "Testis", "Ovary", "Prostate"]
MODE2_CELL_LINE = ""  #@param ["", "HEK293T", "HeLa", "K562", "A549", "MCF7", "U2OS", "HepG2", "SH-SY5Y"]

print("Running translational demand analysis...")
print("  Weights each codon by TPM × number of codons per gene")

demand_kwargs = dict(species=SPECIES_NAME, gene_ids=gene_ids, seed=42)
# Use mode-specific override if set, otherwise fall back to cell 3 values
t = MODE2_TISSUE if MODE2_TISSUE else (tissue if tissue else None)
cl = MODE2_CELL_LINE if MODE2_CELL_LINE else (cell_line if cell_line else None)
if t:
    demand_kwargs["tissue"] = t
if cl:
    demand_kwargs["cell_line"] = cl

demand_result = run_demand(**demand_kwargs)

print(f"\n{demand_result['n_genes']} genes analysed")
print(f"Expression source: {demand_result.get('tissue', 'default')}")

df_demand = demand_result["results"]
n_sig = len(df_demand[df_demand["adjusted_p"] < 0.05])
print(f"{n_sig} codons with significant demand bias (adjusted p < 0.05)")

print("\nTop 20 codons by |Z-score|:")
top_demand = df_demand.head(20).copy()
display(top_demand.round(4))

if "top_genes" in demand_result and demand_result["top_genes"] is not None:
    print("\nTop demand-contributing genes (Demand % = share of total translational output):")
    display(demand_result["top_genes"].head(10))

In [ ]:
#@title Mode 3: Optimality Profile { display-mode: "form" }

import matplotlib.pyplot as plt
import numpy as np
from codonscope.modes.mode3_profile import run_profile

# Number of codons from the 5' end to define the "ramp" region (typically 30-50)
RAMP_CODONS = 50  #@param {type:"integer"}
# wtai = wobble-penalized tRNA Adaptation Index (penalises wobble decoding 0.5x)
# tai = standard tAI (treats all anticodon-codon pairings equally)
METHOD = "wtai"  #@param ["wtai", "tai"]

print(f"Running optimality profile (method={METHOD}, ramp={RAMP_CODONS} codons)...")
if METHOD == "wtai":
    print("  wtAI: higher score = more tRNA gene copies + Watson-Crick decoding preferred")
else:
    print("  tAI: higher score = more tRNA gene copies (no wobble penalty)")

profile_result = run_profile(
    species=SPECIES_NAME,
    gene_ids=gene_ids,
    ramp_codons=RAMP_CODONS,
    method=METHOD,
    seed=42,
)

print(f"\n{profile_result['n_genes']} genes analysed")

# Plot metagene profile
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

mg_gs = profile_result["metagene_geneset"]
mg_bg = profile_result["metagene_genome"]
x = np.arange(len(mg_gs))

ax1.plot(x, mg_gs, label="Gene set", linewidth=2)
ax1.plot(x, mg_bg, label="Genome", linewidth=1, alpha=0.7)
ax1.set_xlabel("Normalised position (%)")
ax1.set_ylabel(f"{METHOD.upper()} score")
ax1.set_title("Metagene Optimality Profile")
ax1.legend()
ax1.axvline(x=RAMP_CODONS * 100 / max(len(mg_gs), 1), color="gray", linestyle="--", alpha=0.5, label="Ramp boundary")

# Ramp analysis summary
ramp = profile_result["ramp_analysis"]
categories = ["Ramp\n(gene set)", "Body\n(gene set)", "Ramp\n(genome)", "Body\n(genome)"]
values = [ramp["ramp_mean_geneset"], ramp["body_mean_geneset"],
          ramp["ramp_mean_genome"], ramp["body_mean_genome"]]
colors = ["#1f77b4", "#1f77b4", "#ff7f0e", "#ff7f0e"]
bars = ax2.bar(categories, values, color=colors, alpha=[0.6, 1, 0.6, 1])
ax2.set_ylabel(f"Mean {METHOD.upper()}")
ax2.set_title("Ramp vs Body Optimality")

plt.tight_layout()
plt.show()

print(f"\nRamp analysis (ramp = first {RAMP_CODONS} codons, body = rest):")
print(f"  Gene set ramp: {ramp['ramp_mean_geneset']:.4f}, body: {ramp['body_mean_geneset']:.4f}")
print(f"  Genome   ramp: {ramp['ramp_mean_genome']:.4f}, body: {ramp['body_mean_genome']:.4f}")
delta = ramp['body_mean_geneset'] - ramp['ramp_mean_geneset']
print(f"  Ramp delta (body - ramp): {delta:+.4f} {'(ramp is slower → expected for highly-expressed genes)' if delta > 0 else ''}")

if profile_result.get("ramp_composition") is not None:
    print("\nRamp codon composition (top 10 by frequency difference):")
    display(profile_result["ramp_composition"].head(10))

In [ ]:
#@title Mode 4: Collision Potential { display-mode: "form" }

from codonscope.modes.mode4_collision import run_collision

print("Running collision potential analysis...")

collision_result = run_collision(
    species=SPECIES_NAME,
    gene_ids=gene_ids,
)

print(f"\n{collision_result['n_genes']} genes analysed")
print(f"FS enrichment ratio: {collision_result['fs_enrichment']:.3f}")
print(f"  (>1.0 = more fast→slow transitions than expected)")
print(f"Chi-squared p-value: {collision_result['chi2_p']:.2e}")

# Transition matrix
print("\nTransition proportions (gene set vs genome):")
tm_gs = collision_result["transition_matrix_geneset"]
tm_bg = collision_result["transition_matrix_genome"]
trans_df = pd.DataFrame({
    "Transition": ["FF", "FS", "SF", "SS"],
    "Gene set": [tm_gs.get(t, 0) for t in ["FF", "FS", "SF", "SS"]],
    "Genome": [tm_bg.get(t, 0) for t in ["FF", "FS", "SF", "SS"]],
}).set_index("Transition")
display(trans_df.round(4))

# Per-dicodon FS enrichment
if collision_result.get("fs_dicodons") is not None and len(collision_result["fs_dicodons"]) > 0:
    print("\nTop FS dicodons (most enriched fast→slow transitions):")
    display(collision_result["fs_dicodons"].head(15).round(4))

In [ ]:
#@title Mode 5: AA vs Codon Disentanglement { display-mode: "form" }

from codonscope.modes.mode5_disentangle import run_disentangle

print("Running disentanglement analysis...")

dis_result = run_disentangle(
    species=SPECIES_NAME,
    gene_ids=gene_ids,
    seed=42,
)

summary = dis_result["summary"]
print(f"\n{dis_result['n_genes']} genes analysed")
print(f"\nSignificant codons: {summary['n_significant_codons']}")
print(f"  AA-driven:          {summary['n_aa_driven']} ({summary['pct_aa_driven']:.0f}%)")
print(f"  Synonymous-driven:  {summary['n_synonymous_driven']} ({summary['pct_synonymous_driven']:.0f}%)")
print(f"  Both:               {summary['n_both']} ({summary['pct_both']:.0f}%)")
print(f"\n{summary['summary_text']}")

# Attribution table
print("\nPer-codon attribution:")
attr = dis_result["attribution"].copy()
attr_display = attr[["codon", "amino_acid", "aa_z_score", "rscu_z_score", "attribution"]].copy()
attr_display = attr_display.round({"aa_z_score": 2, "rscu_z_score": 2})

def color_attribution(val):
    colors = {
        "AA-driven": "background-color: #cce5ff",
        "Synonymous-driven": "background-color: #d4edda",
        "Both": "background-color: #fff3cd",
        "None": "",
    }
    return colors.get(val, "")

display(attr_display.style.applymap(color_attribution, subset=["attribution"]))

# Synonymous drivers
syn = dis_result["synonymous_drivers"]
syn_sig = syn[syn["driver"] != "not_applicable"]
if len(syn_sig) > 0:
    print("\nSynonymous driver classification:")
    display(syn_sig[["codon", "amino_acid", "rscu_z_score", "driver"]].round(2))

In [ ]:
#@title Mode 6: Cross-Species Comparison (requires orthologs) { display-mode: "form" }

if not species2:
    print("Skipped: no second species selected in cell 3.")
    print("Set SPECIES2 above and re-run to enable cross-species comparison.")
else:
    from codonscope.modes.mode6_compare import run_compare

    print(f"Running cross-species comparison: {SPECIES_NAME} vs {species2}...")

    compare_result = run_compare(
        species1=SPECIES_NAME,
        species2=species2,
        gene_ids=gene_ids,
        seed=42,
    )

    s = compare_result["summary"]
    print(f"\n{compare_result['n_orthologs']} ortholog pairs found in gene set")
    print(f"{compare_result['n_genome_orthologs']} ortholog pairs genome-wide")
    print(f"\nMean RSCU correlation:")
    print(f"  Gene set: {s['mean_r_geneset']:.3f}")
    print(f"  Genome:   {s['mean_r_genome']:.3f}")
    print(f"  Z-score:  {s['z_score']:.2f}")
    print(f"  P-value:  {s['p_value']:.2e}")

    # Per-gene correlations
    print("\nPer-gene RSCU correlations:")
    per_gene = compare_result["per_gene"].sort_values("r", ascending=False)
    display(per_gene.head(20).round(3))

    # Divergent genes
    if compare_result.get("divergent_analysis") is not None and len(compare_result["divergent_analysis"]) > 0:
        print("\nMost divergent genes (different preferred codons):")
        display(compare_result["divergent_analysis"].head(10).round(3))

---

## Tips

- **Low Z-scores for highly expressed genes?** This is expected when your gene set dominates the genome's translational demand (e.g., ribosomal proteins in yeast make up ~72% of demand). The gene set IS the background.
- **Matched vs all background:** Use `matched` in Mode 1 to control for CDS length and GC content. This removes compositional confounds and reveals true codon selection.
- **Tricodon analysis:** Requires large gene sets (>50 genes) for statistical power. 226,981 possible tricodons means heavy multiple testing correction.
- **Cross-species comparison:** Low RSCU correlation between orthologs is biologically meaningful — it indicates different tRNA pools drive different codon preferences.
- **Custom expression data:** Use `--expression-file` in the CLI or the `expression_file` parameter to supply your own TPM values for demand analysis.